<a href="https://colab.research.google.com/github/RAVIPATISRIVIDYA/devtraining-needit-madrid/blob/master/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, AvgPool1D, Dense, Softmax
from tensorflow.keras.datasets import imdb

In [ ]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [ ]:
from sklearn.model_selection import train_test_split

x_train_sub, _,y_train_sub,_ = train_test_split(x_train, y_train, stratify=y_train, train_size=.08, shuffle = True,)
x_test_sub, _, y_test_sub,_ = train_test_split(x_test, y_test, stratify=y_train, train_size=.004, shuffle = True,)

print('Training set size:',x_train_sub.shape)
print('Test set size',x_test_sub.shape)


Training set size: (2000, 80)
Test set size (100, 80)


In [ ]:
word_dictionary = imdb.get_word_index()

In [ ]:
index_to_word_dictionary = dict(zip(word_dictionary.values(),word_dictionary.keys()))

In [ ]:
def index_to_word(index):
  if (index > 0) and (index < len(word_dictionary)):
    return index_to_word_dictionary.get(index)
    return 'not in dictionary'
def show_sentence(x):
  for j in x:
    print(index_to_word(j),' ', end='')
  print()

In [ ]:
e = Embedding(max_features, 80, input_length=80,mask_zero=True)
l = LSTM(80, dropout=0.2, recurrent_dropout=0.2)
d = Dense(1, activation='sigmoid') 

model = Sequential([e,l,d])
#model.add(Softmax())

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 80, 80)            1600000   
                                                                 
 lstm_6 (LSTM)               (None, 80)                51520     
                                                                 
 dense_6 (Dense)             (None, 1)                 81        
                                                                 
Total params: 1,651,601
Trainable params: 1,651,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print('Training...')

model.fit(x_train_sub, y_train_sub,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test_sub, y_test_sub),
          callbacks = [])

Training...
Epoch 1/3
63/63 [==============================] - 16s 191ms/step - loss: 0.6595 - accuracy: 0.5895 - val_loss: 0.5822 - val_accuracy: 0.6800
Epoch 2/3
63/63 [==============================] - 12s 183ms/step - loss: 0.3851 - accuracy: 0.8560 - val_loss: 0.3668 - val_accuracy: 0.8500
Epoch 3/3
63/63 [==============================] - 12s 184ms/step - loss: 0.1276 - accuracy: 0.9610 - val_loss: 0.4064 - val_accuracy: 0.8400


In [ ]:
score, acc = model.evaluate(x_test_sub, y_test_sub,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

4/4 [==============================] - 0s 26ms/step - loss: 0.4064 - accuracy: 0.8400
Test score: 0.4063774049282074
Test accuracy: 0.8399999737739563


In [ ]:
def jacobian_for(x):
  """
  Calculate jacobian of logits wrt input.
  """
  x = tf.convert_to_tensor(x,dtype=tf.float32)
  x = tf.expand_dims(x,0)
  with tf.GradientTape(watch_accessed_variables=False, persistent=True) as tape:
    tape.watch(e.variables)
    y = model(x)
    return tf.squeeze(tape.jacobian(y, e.variables, parallel_iterations=100, experimental_use_pfor=False))

In [ ]:
def convert_to_label(y):
  if y[0]<=.5:
    y = 0
  else:
    y = 1
  return y


def imdb_attack(f, x, D=np.array(list(word_dictionary.values()),dtype='int64'), max_iter=100):

  """performs attack proposed in Papernot et
al, Crafting Adversarial Input Sequences for Recurrent
Neural Networks."""

  y = f.predict(x)

  y = convert_to_label(y)

  x_ad = x.copy()

  ad_prediction = convert_to_label(f.predict(x_ad))

  jacobian = jacobian_for(np.squeeze(x))

  iteration = 0

  while ad_prediction == y:

    iteration += 1

    if iteration > max_iter:
      break

    #print('\niteration:',iteration)

    i = np.random.randint(0,80,1)[0]

    #print('i:',i)

    #==================TODO==============================
    # 1. Figure out function for w as described in paper
    #
    #====================================================
    j_x = jacobian[i]

    z = tf.argmin(np.sign(D - x_ad[0][i])) #+ 1 #b/c dictionary starts on index 1

    w = np.abs(z - np.sign(j_x[i]))


    #================================================
    #================================================
   
    #=======DELETE WHEN FUNCTION IS FULLY IMPLEMENTED
    if w==None: break 
    #=======DELETE WHEN FUNCTION IS FULLY IMPLEMENTED
    #print('old word:',x_ad[0][i],index_to_word(x_ad[0][i]))

    if not w < max_features:
      continue

    #print('new word:',w,index_to_word(w))

    
    x_ad[0][i]=w

    ad_prediction = convert_to_label(f.predict(x_ad))

  #return adversarial example

  if ad_prediction != y:
    print(' NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN',iteration,'ITERATIONS')

  return x_ad

In [ ]:
adversarial_example = np.array([imdb_attack(model,np.expand_dims(i,0)) for i in x_test_sub]).squeeze()

 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 80 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 22 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 11 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 16 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 78 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 55 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 61 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 86 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 41 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 67 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 58 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 5 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 70 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 44 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 3 ITERATIONS
 NEW ADVERSARIAL EXAMPLE SUCCESSFULLY CREATED IN 72 ITERA

In [ ]:
for i in range(len(adversarial_example)):
  print('Sentence ',i,'Original')
  show_sentence(x_test_sub[i])
  print('\nSentence ',i,'Adversarial')
  show_sentence(adversarial_example[i])
  print('===========================================================================================================================================================\n')

Sentence  0 Original
does  of  feel  features  christmas  kind  so  paid  not  contemplating  horror  in  at  of  its  others  this  of  script  their  valuable  or  accused  generally  takes  laughs  seems  parallel  of  lyric  hold  resorts  of  following  it's  with  that  out  is  plus  interested  have  best  does  still  can  costume  and  but  cinematography  two  i  i  him  love  br  an  of  down  then  poorly  in  laughed  of  making  if  times  this  is  put  quite  br  played  good  website  problems  br  didn't  watch  an  

Sentence  0 Adversarial
becomes  of  those  people  into  real  laugh  paid  already  to  other  animate  at  of  its  others  this  of  laugh  people  to  laughable  to  becomes  real  into  real  to  of  to  into  resorts  of  following  people  keyboard  that  people  is  plus  people  mistake  into  does  becomes  can  to  and  people  laugh  becomes  i  i  people  becomes  br  an  of  becomes  people  becomes  in  to  of  real  if  becomes  stills 

In [ ]:
print(model.evaluate(x_test_sub,y_test_sub,batch_size=batch_size))
print(model.evaluate(adversarial_example, y_test_sub, batch_size=batch_size))

4/4 [==============================] - 0s 22ms/step - loss: 0.4064 - accuracy: 0.8400
[0.4063774049282074, 0.8399999737739563]
4/4 [==============================] - 0s 26ms/step - loss: 0.6169 - accuracy: 0.5600
[0.6169216632843018, 0.5600000023841858]
